In [186]:
%xmode plain
import glob
import cv2
import numpy as np

Exception reporting mode: Plain


In [187]:
suits_dict = {0:'Diamond',1:'Spade', 2:'Heart', 3:'Club'}
ranks_dict = {0:'A', 1:'2', 2:'3', 3:'4', 4:'5', 5:'6', 6:'7', 7:'8', 8:'9', 9:'10', 10:'J', 11:'K', 12:'Q'}
BKG_THRESH = 60
CARD_THRESH = 30

In [188]:
def flattener(image, pts, w, h):
    """Flattens an image of a card into a top-down 200x300 perspective.
    Returns the flattened, re-sized, grayed image.
    See www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/"""
    temp_rect = np.zeros((4,2), dtype = "float32")
    
    s = np.sum(pts, axis = 2)

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis = -1)
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    
#     pri
    # Need to create an array listing points in order of
    # [top left, top right, bottom right, bottom left]
    # before doing the perspective transform

    if w <= 0.8*h: # If card is vertically oriented
        temp_rect[0] = tl
        temp_rect[1] = tr
        temp_rect[2] = br
        temp_rect[3] = bl

    if w >= 1.2*h: # If card is horizontally oriented
        temp_rect[0] = bl
        temp_rect[1] = tl
        temp_rect[2] = tr
        temp_rect[3] = br

    # If the card is 'diamond' oriented, a different algorithm
    # has to be used to identify which point is top left, top right
    # bottom left, and bottom right.
    
    if w > 0.8*h and w < 1.2*h: #If card is diamond oriented
        # If furthest left point is higher than furthest right point,
        # card is tilted to the left.
        if pts[1][0][1] <= pts[3][0][1]:
            # If card is titled to the left, approxPolyDP returns points
            # in this order: top right, top left, bottom left, bottom right
            temp_rect[0] = pts[1][0] # Top left
            temp_rect[1] = pts[0][0] # Top right
            temp_rect[2] = pts[3][0] # Bottom right
            temp_rect[3] = pts[2][0] # Bottom left

        # If furthest left point is lower than furthest right point,
        # card is tilted to the right
        if pts[1][0][1] > pts[3][0][1]:
            # If card is titled to the right, approxPolyDP returns points
            # in this order: top left, bottom left, bottom right, top right
            temp_rect[0] = pts[0][0] # Top left
            temp_rect[1] = pts[3][0] # Top right
            temp_rect[2] = pts[2][0] # Bottom right
            temp_rect[3] = pts[1][0] # Bottom left
            
        
    maxWidth = 200
    maxHeight = 300

    # Create destination array, calculate perspective transform matrix,
    # and warp card image
    dst = np.array([[0,0],[maxWidth-1,0],[maxWidth-1,maxHeight-1],[0, maxHeight-1]], np.float32)
    M = cv2.getPerspectiveTransform(temp_rect,dst)
    warp = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    return warp

In [189]:
def preprocess_card(contour, img):
    
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.01*peri, True)
    pts = np.float32(approx)
    x,y,w,h = cv2.boundingRect(approx)
    
    warp = flattener(img, pts, w, h)
    
    cv2.imshow('warp', warp)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    ## Card image
    warp = cv2.cvtColor(warp,cv2.COLOR_BGR2GRAY)

    CORNER_WIDTH = 30
    CORNER_HEIGHT = 84
    
    cropped_suit = warp[0:CORNER_HEIGHT, 0:CORNER_WIDTH]
    csuit = cv2.resize(cropped_suit, (0,0), fx=4, fy=4)
    
    white_level = csuit[15,int((CORNER_WIDTH*4)/2)]
    thresh_level = white_level - CARD_THRESH
    
    if (thresh_level <= 0):
        thresh_level = 1
    
    print("Threeshold value: ", thresh_level)

    half = int(csuit.shape[0]/2)

    retval, rank = cv2.threshold(csuit[0:half,:], thresh_level, 255, cv2. THRESH_BINARY_INV)
    retval, suit = cv2.threshold(csuit[half+1:half*2,:], thresh_level, 255, cv2. THRESH_BINARY_INV)
    
    return rank , suit

In [190]:
def sift_detect(cropped_suit, suits, suit_dict):
    sift = cv2.xfeatures2d.SIFT_create()
    
    ########################
    for i,img in enumerate(suits):
        _,img = cv2.threshold(img,128,255,cv2.THRESH_BINARY)
        suits[i] = img
    ########################

    
    distances = []
    for i in range(len(suits)):
        kp1, des1 = sift.detectAndCompute(cropped_suit, None)
        sift = cv2.xfeatures2d.SIFT_create()
        kp2, des2 = sift.detectAndCompute(suits[i], None)
        bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
        matches = bf.match(des1, des2)
        matches = sorted(matches, key = lambda x:x.distance)
        sums = 0
        for j in range(3):
            sums += matches[j].distance
        distances.append(sums)
    print(" The card suit is:  ", suit_dict[np.argmin(np.array(distances))])
    

In [191]:
def find_suit(img_suit):
    suits = [cv2.imread(file) for file in glob.glob("suits1/*.png")]

    for i,img in enumerate(suits):
        img = cv2.cvtColor(cv2.resize(img, (img_suit.shape[1], img_suit.shape[0])), cv2.COLOR_BGR2GRAY)
        _,img = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
        suits[i] = img
        
    minimum2 = 1000000
    idx = 0
    
    for j in range(len(suits)):
        ans = np.abs(-img_suit + suits[j])
        ans = ans[ans > 128].shape[0]
#         print(j," - ", ans)
        if ans < minimum2:
            minimum2 = ans
            idx = j
    return idx

In [192]:
def find_rank(rank, train_ranks):
    
    for i,img in enumerate(train_ranks):
        _,img = cv2.threshold(img,175,255,cv2.THRESH_BINARY)
        train_ranks[i] = img
        
    minimum2 = 1000000
    idx = 0
    
    for j in range(len(train_ranks)):    
        ans = np.abs(-rank + train_ranks[j])
        ans = ans[ans > 128].shape[0]
#         print(j," - ", ans)
        if ans < minimum2:
            minimum2 = ans
            idx = j
            
    return idx

In [238]:
mask = cv2.imread('../CV-BlackjackStudent/GOPR0319.MP4-lbl/13123-lbl.png',1)
input_img = cv2.imread('../CV-BlackjackStudent/GOPR0319.MP4-lbl/13123.png',1)
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(mask,128,255,0)
_, input_img_t = cv2.threshold(input_img,128,255,0)
im2, num_cards, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)


In [246]:
for i in range(len(num_cards)):
    
    card_value, card_suit = preprocess_card(num_cards[i], input_img)
    im2, contours, hierarchy = cv2.findContours(card_value,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea,reverse=True)

    x, y, w, h = cv2.boundingRect(contours[0])
    isolated_rank = card_value[y:y+h, x:x+w]
    im2, contours, hierarchy = cv2.findContours(card_suit,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea,reverse=True)
    peri = cv2.arcLength(contours[0], True)
    approx = cv2.approxPolyDP(contours[0], 0.09*peri, True)
    pts = np.float32(approx)
    x,y,w,h = cv2.boundingRect(approx)
    # x, y, w, h = cv2.boundingRect(contours[0])
    isolated_suit = card_suit[y:y+h, x:x+w]
    
    ## Read in card suits

    suits = [cv2.imread(file) for file in glob.glob("suits/*.png")]
    for i,img in enumerate(suits):
        img = cv2.cvtColor(cv2.resize(img, (isolated_suit.shape[1], isolated_suit.shape[0])), cv2.COLOR_BGR2GRAY)
        suits[i] = img

    ## Read in the card values
    arr = ['A','2','3','4','5','6','7','8','9','10','J','K','Q']
    ranks = [cv2.imread('values/'+file+'.png',1) for file in arr]
    for i,img in enumerate(ranks):
        img = cv2.cvtColor(cv2.resize(img, (isolated_rank.shape[1], isolated_rank.shape[0])), cv2.COLOR_BGR2GRAY)
        ranks[i] = img

    cv2.imshow('query suit', isolated_suit)
    cv2.imshow('query rank', isolated_rank)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    a = find_suit(isolated_suit)
    b = find_rank(isolated_rank, ranks)

    print("The suit of the card is: ", suits_dict[a], " value: ", ranks_dict[b])

Threeshold value:  196
The suit of the card is:  Diamond  value:  3
Threeshold value:  188
The suit of the card is:  Diamond  value:  7
Threeshold value:  180
The suit of the card is:  Club  value:  J


In [195]:
## Read in card suits

suits = [cv2.imread(file) for file in glob.glob("suits/*.png")]
for i,img in enumerate(suits):
    img = cv2.cvtColor(cv2.resize(img, (isolated_suit.shape[1], isolated_suit.shape[0])), cv2.COLOR_BGR2GRAY)
    suits[i] = img
        
## Read in the card values
arr = ['A','2','3','4','5','6','7','8','9','10','J','K','Q']
ranks = [cv2.imread('values/'+file+'.png',1) for file in arr]
for i,img in enumerate(ranks):
    img = cv2.cvtColor(cv2.resize(img, (isolated_rank.shape[1], isolated_rank.shape[0])), cv2.COLOR_BGR2GRAY)
    ranks[i] = img
    
cv2.imshow('query suit', isolated_suit)
cv2.imshow('query rank', isolated_rank)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [196]:
a = find_suit(isolated_suit)
b = find_rank(isolated_rank, ranks)

print("The suit of the card is: ", suits_dict[a], " value: ", ranks_dict[b])

# print("------ SIFT ------")
# sift_detect(isolated_suit, suits, suits_dict)
# sift_detect(isolated_rank, ranks, ranks_dict)

The suit of the card is:  Club  value:  4
